In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
standard_info = pd.read_csv('./standard_info.csv')
standard_info.drop_duplicates(inplace=True)
pd.set_option('display.max_columns',None)
game_info = pd.read_csv('game_info.csv')
standard_info = pd.merge(standard_info,game_info,on='game_id',how='left',validate='many_to_one')

C:\Users\Administrator\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
standard_info_an = standard_info[standard_info['game_platform']==1]
standard_info_ios = standard_info[standard_info['game_platform']==2]

In [36]:
def standard_compute_an(standard_info_an):
    # 用户质量维度
    source_info_2_grade_1_user = pd.DataFrame(columns=['source_run_7date_bigrate','source_run_7date_midrate','source_run_7date_minrate',
                                                      'source_run_7date_comrate','source_run_7date_rubrate'])
    source_info_2_grade_2_user = pd.DataFrame(columns=['source_run_7date_bigrate','source_run_7date_midrate','source_run_7date_minrate',
                                                      'source_run_7date_comrate','source_run_7date_rubrate'])
    source_info_2_grade_3_user = pd.DataFrame(columns=['source_run_7date_bigrate','source_run_7date_midrate','source_run_7date_minrate',
                                                      'source_run_7date_comrate','source_run_7date_rubrate'])

    standard_info_duplicates_user = standard_info_an[standard_info_an['role_data_type'] == 7].drop_duplicates(['user_id','cp_server_no','cp_role_id'])
#     standard_info_duplicates_user = standard_info_duplicates_user[standard_info_duplicates_user['source_amount']!=0]
    standard_info_duplicates_user['b_r'] = (standard_info_duplicates_user['role_created_30_pay_sum']>=2000).astype(int)
    standard_info_duplicates_user['m_r'] = ((standard_info_duplicates_user['role_created_30_pay_sum']<2000) & (standard_info_duplicates_user['role_created_30_pay_sum']>=100)).astype(int)
    standard_info_duplicates_user['s_r'] = ((standard_info_duplicates_user['role_created_30_pay_sum']<100) & (standard_info_duplicates_user['role_created_30_pay_sum']>0)).astype(int)
    standard_info_duplicates_user['rubbish'] = ((standard_info_duplicates_user['role_created_7_online']<=600) & (standard_info_duplicates_user['role_created_7_login_num']<=2)&
                                (standard_info_duplicates_user['role_created_7_max_role_level']<=10)&(standard_info_duplicates_user['role_created_30_pay_sum']==0)).astype(int)
    standard_info_duplicates_user['common'] = (~((standard_info_duplicates_user['role_created_7_online']<=600) & (standard_info_duplicates_user['role_created_7_login_num']<=2)&
                (standard_info_duplicates_user['role_created_7_max_role_level']<=10)) &(standard_info_duplicates_user['role_created_30_pay_sum']==0)).astype(int)
    groups = standard_info_duplicates_user.groupby(['channel_id','source_id','create_role_date'])

    temp = groups['role_created_30_pay_sum'].agg([('source_30_pay_sum','sum')]).reset_index()
    standard_info_duplicates_user = pd.merge(standard_info_duplicates_user,temp,on=['channel_id','source_id','create_role_date'],how='left',validate='many_to_one')

    source_grade_1_user = standard_info_duplicates_user[standard_info_duplicates_user['source_amount']>=5000]
    source_grade_2_user = standard_info_duplicates_user[(standard_info_duplicates_user['source_amount']<5000)&(standard_info_duplicates_user['source_amount']>=3000)]
    source_grade_3_user = standard_info_duplicates_user[(standard_info_duplicates_user['source_amount']<3000)&(standard_info_duplicates_user['source_amount']>=500)]
    
    # grade_1
    if len(source_grade_1_user)!=0:

        source_run_7date_bigrate_grade_1 = round(source_grade_1_user['b_r'].sum()/source_grade_1_user.shape[0],4)
        source_run_7date_midrate_grade_1 = round(source_grade_1_user['m_r'].sum()/source_grade_1_user.shape[0],4)
        source_run_7date_minrate_grade_1 = round(source_grade_1_user['s_r'].sum()/source_grade_1_user.shape[0],4)
        source_run_7date_comrate_grade_1 = round(source_grade_1_user['common'].sum()/source_grade_1_user.shape[0],4)
        source_run_7date_rubrate_grade_1 = round(source_grade_1_user['rubbish'].sum()/source_grade_1_user.shape[0],4)
   
    # grade_2
    if len(source_grade_2_user)!=0:
        source_run_7date_bigrate_grade_2 = round(source_grade_2_user['b_r'].sum()/source_grade_2_user.shape[0],4)
        source_run_7date_midrate_grade_2 = round(source_grade_2_user['m_r'].sum()/source_grade_2_user.shape[0],4)
        source_run_7date_minrate_grade_2 = round(source_grade_2_user['s_r'].sum()/source_grade_2_user.shape[0],4)
        source_run_7date_comrate_grade_2 = round(source_grade_2_user['common'].sum()/source_grade_2_user.shape[0],4)
        source_run_7date_rubrate_grade_2 = round(source_grade_2_user['rubbish'].sum()/source_grade_2_user.shape[0],4)
        
    # grade_3
    if len(source_grade_3_user)!=0:
        source_run_7date_bigrate_grade_3 = round(source_grade_3_user['b_r'].sum()/source_grade_3_user.shape[0],4)
        source_run_7date_midrate_grade_3 = round(source_grade_3_user['m_r'].sum()/source_grade_3_user.shape[0],4)
        source_run_7date_minrate_grade_3 = round(source_grade_3_user['s_r'].sum()/source_grade_3_user.shape[0],4)
        source_run_7date_comrate_grade_3 = round(source_grade_3_user['common'].sum()/source_grade_3_user.shape[0],4)
        source_run_7date_rubrate_grade_3 = round(source_grade_3_user['rubbish'].sum()/source_grade_3_user.shape[0],4)
    for i in range(1,4):        
        exec("source_info_2_grade_%s_user['source_run_7date_bigrate'] = [source_run_7date_bigrate_grade_%s]"%(i,i))
        exec("source_info_2_grade_%s_user['source_run_7date_midrate'] = [source_run_7date_midrate_grade_%s]"%(i,i))
        exec("source_info_2_grade_%s_user['source_run_7date_minrate'] = [source_run_7date_minrate_grade_%s]"%(i,i))
        exec("source_info_2_grade_%s_user['source_run_7date_comrate'] = [source_run_7date_comrate_grade_%s]"%(i,i))
        exec("source_info_2_grade_%s_user['source_run_7date_rubrate'] = [source_run_7date_rubrate_grade_%s]"%(i,i))
    
    
    source_info_2_user = pd.concat([source_info_2_grade_1_user,source_info_2_grade_2_user,source_info_2_grade_3_user]).reset_index(drop=True)        
    
    
    
    # 计划成本、ROI维度
    source_info_2_grade_1 = pd.DataFrame(columns=['source_cost','standard_run_date','game_platform','source_run_date_cost','source_run_date_roi_1',
                                                      'source_run_date_roi_2','source_run_date_roi_3','source_run_date_roi_4','source_run_date_roi_5',
                                                      'source_run_date_roi_6','source_run_date_roi_7','source_run_date_predict_roi_30'])
    source_info_2_grade_2 = pd.DataFrame(columns=['source_cost','standard_run_date','game_platform','source_run_date_cost','source_run_date_roi_1',
                                                      'source_run_date_roi_2','source_run_date_roi_3','source_run_date_roi_4','source_run_date_roi_5',
                                                      'source_run_date_roi_6','source_run_date_roi_7','source_run_date_predict_roi_30'])
    source_info_2_grade_3 = pd.DataFrame(columns=['source_cost','standard_run_date','game_platform','source_run_date_cost','source_run_date_roi_1',
                                                      'source_run_date_roi_2','source_run_date_roi_3','source_run_date_roi_4','source_run_date_roi_5',
                                                      'source_run_date_roi_6','source_run_date_roi_7','source_run_date_predict_roi_30'])
    
    
    for i in range(1,8):
        standard_info_duplicates = standard_info_an[standard_info_an['role_data_type'] == i].drop_duplicates(['user_id','cp_server_no','cp_role_id'])
        # standard_info_duplicates = standard_info_duplicates[standard_info_duplicates['source_amount']!=0]
        
        source_grade_1 = standard_info_duplicates[standard_info_duplicates['source_amount']>=5000]
        source_grade_2 = standard_info_duplicates[(standard_info_duplicates['source_amount']<5000)&(standard_info_duplicates['source_amount']>=3000)]
        source_grade_3 = standard_info_duplicates[(standard_info_duplicates['source_amount']<3000)&(standard_info_duplicates['source_amount']>=500)]
        
        
        def dell(df):
            groups = df.groupby(['channel_id','source_id'])
            temp = groups['pay_sum'].agg([('pay_sum_nd','sum')]).reset_index()
            df = pd.merge(df,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')

            temp = groups['role_created_30_pay_sum'].agg([('source_30_pay_sum','sum')]).reset_index()
            df = pd.merge(df,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')

            temp = groups['user_id'].agg([('user_num','count')]).reset_index()
            df = pd.merge(df,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
            df = df[['channel_id','source_id','create_role_date','source_amount','game_platform',
                                                         'pay_sum_nd','source_30_pay_sum','user_num']].drop_duplicates()
            groups = df.groupby(['channel_id','source_id'])
            temp = groups['source_amount'].agg([('source_amount_sum','sum')]).reset_index()
            df = pd.merge(df,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
            df = df[['channel_id','source_id','source_amount_sum','game_platform',
                                                          'pay_sum_nd','source_30_pay_sum','user_num']].drop_duplicates()
            return df
        source_grade_1 = dell(source_grade_1)
        source_grade_2 = dell(source_grade_2)
        source_grade_3 = dell(source_grade_3)
                
        # grade_1
        if len(source_grade_1)!=0:             
            source_run_date_cost_grade_1 = round(source_grade_1[source_grade_1['user_num']!=0]['source_amount_sum'].sum()/source_grade_1[source_grade_1['user_num']!=0]['user_num'].sum(),2)            
            source_run_date_predict_roi_30_grade_1 = round(source_grade_1['source_30_pay_sum'].sum()/source_grade_1['source_amount_sum'].sum(),6)
            exec("source_run_date_roi_%s_grade_1 = round(source_grade_1['pay_sum_nd'].sum()/source_grade_1['source_amount_sum'].sum(),6)"%(i))
                      
        # grade_2
        if len(source_grade_2)!=0:
            source_run_date_cost_grade_2 = round(source_grade_2[source_grade_2['user_num']!=0]['source_amount_sum'].sum()/source_grade_2[source_grade_2['user_num']!=0]['user_num'].sum(),2)  
            source_run_date_predict_roi_30_grade_2 = round(source_grade_2['source_30_pay_sum'].sum()/source_grade_2['source_amount_sum'].sum(),6)
            exec("source_run_date_roi_%s_grade_2 = round(source_grade_2['pay_sum_nd'].sum()/source_grade_2['source_amount_sum'].sum(),6)"%(i))
         
        # grade_3
        if len(source_grade_3)!=0:
            source_run_date_cost_grade_3 = round(source_grade_3[source_grade_3['user_num']!=0]['source_amount_sum'].sum()/source_grade_3[source_grade_3['user_num']!=0]['user_num'].sum(),2)
            source_run_date_predict_roi_30_grade_3 = round(source_grade_3['source_30_pay_sum'].sum()/source_grade_3['source_amount_sum'].sum(),6)
            exec("source_run_date_roi_%s_grade_3 = round(source_grade_3['pay_sum_nd'].sum()/source_grade_3['source_amount_sum'].sum(),6)"%(i))
                                             
        source_info_2_grade_1['game_platform'] = [1]
        source_info_2_grade_1['source_run_date_cost'] = [source_run_date_cost_grade_1]
        exec("source_info_2_grade_1['source_run_date_roi_%s'] = [source_run_date_roi_%s_grade_1]"%(i,i))
        source_info_2_grade_1['source_run_date_predict_roi_30'] = [source_run_date_predict_roi_30_grade_1]
        source_info_2_grade_1['source_cost'] = ['>=5000']
        source_info_2_grade_1['standard_run_date'] = [datetime.date.today()]
   
        source_info_2_grade_2['game_platform'] = [1]
        source_info_2_grade_2['source_run_date_cost'] = [source_run_date_cost_grade_2]
        exec("source_info_2_grade_2['source_run_date_roi_%s'] = [source_run_date_roi_%s_grade_2]"%(i,i))
        source_info_2_grade_2['source_run_date_predict_roi_30'] = [source_run_date_predict_roi_30_grade_2]
        source_info_2_grade_2['source_cost'] = ['<5000 and >=3000']
        source_info_2_grade_2['standard_run_date'] = [datetime.date.today()]

        source_info_2_grade_3['game_platform'] = [1]
        source_info_2_grade_3['source_run_date_cost'] = [source_run_date_cost_grade_3]
        exec("source_info_2_grade_3['source_run_date_roi_%s'] = [source_run_date_roi_%s_grade_3]"%(i,i))
        source_info_2_grade_3['source_run_date_predict_roi_30'] = [source_run_date_predict_roi_30_grade_3]
        source_info_2_grade_3['source_cost'] = ['<3000 and >=500']
        source_info_2_grade_3['standard_run_date'] = [datetime.date.today()]
    
    source_info_2 = pd.concat([source_info_2_grade_1,source_info_2_grade_2,source_info_2_grade_3]).reset_index(drop=True)
    df = pd.concat([source_info_2,source_info_2_user], axis=1)
    return df

In [37]:
def standard_compute_ios(standard_info_ios):
    # 用户质量维度
    source_info_2_grade_1_user = pd.DataFrame(columns=['source_run_7date_bigrate','source_run_7date_midrate','source_run_7date_minrate',
                                                      'source_run_7date_comrate','source_run_7date_rubrate'])
    source_info_2_grade_2_user = pd.DataFrame(columns=['source_run_7date_bigrate','source_run_7date_midrate','source_run_7date_minrate',
                                                      'source_run_7date_comrate','source_run_7date_rubrate'])
    source_info_2_grade_3_user = pd.DataFrame(columns=['source_run_7date_bigrate','source_run_7date_midrate','source_run_7date_minrate',
                                                      'source_run_7date_comrate','source_run_7date_rubrate'])

    standard_info_duplicates_user = standard_info_ios[standard_info_ios['role_data_type'] == 7].drop_duplicates(['user_id','cp_server_no','cp_role_id'])
#     standard_info_duplicates_user = standard_info_duplicates_user[standard_info_duplicates_user['source_amount']!=0]
    standard_info_duplicates_user['b_r'] = (standard_info_duplicates_user['role_created_30_pay_sum']>=2000).astype(int)
    standard_info_duplicates_user['m_r'] = ((standard_info_duplicates_user['role_created_30_pay_sum']<2000) & (standard_info_duplicates_user['role_created_30_pay_sum']>=100)).astype(int)
    standard_info_duplicates_user['s_r'] = ((standard_info_duplicates_user['role_created_30_pay_sum']<100) & (standard_info_duplicates_user['role_created_30_pay_sum']>0)).astype(int)
    standard_info_duplicates_user['rubbish'] = ((standard_info_duplicates_user['role_created_7_online']<=600) & (standard_info_duplicates_user['role_created_7_login_num']<=2)&
                                (standard_info_duplicates_user['role_created_7_max_role_level']<=10)&(standard_info_duplicates_user['role_created_30_pay_sum']==0)).astype(int)
    standard_info_duplicates_user['common'] = (~((standard_info_duplicates_user['role_created_7_online']<=600) & (standard_info_duplicates_user['role_created_7_login_num']<=2)&
                (standard_info_duplicates_user['role_created_7_max_role_level']<=10)) &(standard_info_duplicates_user['role_created_30_pay_sum']==0)).astype(int)
    groups = standard_info_duplicates_user.groupby(['channel_id','source_id','create_role_date'])

    temp = groups['role_created_30_pay_sum'].agg([('source_30_pay_sum','sum')]).reset_index()
    standard_info_duplicates_user = pd.merge(standard_info_duplicates_user,temp,on=['channel_id','source_id','create_role_date'],how='left',validate='many_to_one')

    source_grade_1_user = standard_info_duplicates_user[standard_info_duplicates_user['source_amount']>=10000]
    source_grade_2_user = standard_info_duplicates_user[(standard_info_duplicates_user['source_amount']<10000)&(standard_info_duplicates_user['source_amount']>=5000)]
    source_grade_3_user = standard_info_duplicates_user[(standard_info_duplicates_user['source_amount']<5000)&(standard_info_duplicates_user['source_amount']>=500)]
    
    # grade_1
    if len(source_grade_1_user)!=0:

        source_run_7date_bigrate_grade_1 = round(source_grade_1_user['b_r'].sum()/source_grade_1_user.shape[0],4)
        source_run_7date_midrate_grade_1 = round(source_grade_1_user['m_r'].sum()/source_grade_1_user.shape[0],4)
        source_run_7date_minrate_grade_1 = round(source_grade_1_user['s_r'].sum()/source_grade_1_user.shape[0],4)
        source_run_7date_comrate_grade_1 = round(source_grade_1_user['common'].sum()/source_grade_1_user.shape[0],4)
        source_run_7date_rubrate_grade_1 = round(source_grade_1_user['rubbish'].sum()/source_grade_1_user.shape[0],4)
   
    # grade_2
    if len(source_grade_2_user)!=0:
        source_run_7date_bigrate_grade_2 = round(source_grade_2_user['b_r'].sum()/source_grade_2_user.shape[0],4)
        source_run_7date_midrate_grade_2 = round(source_grade_2_user['m_r'].sum()/source_grade_2_user.shape[0],4)
        source_run_7date_minrate_grade_2 = round(source_grade_2_user['s_r'].sum()/source_grade_2_user.shape[0],4)
        source_run_7date_comrate_grade_2 = round(source_grade_2_user['common'].sum()/source_grade_2_user.shape[0],4)
        source_run_7date_rubrate_grade_2 = round(source_grade_2_user['rubbish'].sum()/source_grade_2_user.shape[0],4)
        
    # grade_3
    if len(source_grade_3_user)!=0:
        source_run_7date_bigrate_grade_3 = round(source_grade_3_user['b_r'].sum()/source_grade_3_user.shape[0],4)
        source_run_7date_midrate_grade_3 = round(source_grade_3_user['m_r'].sum()/source_grade_3_user.shape[0],4)
        source_run_7date_minrate_grade_3 = round(source_grade_3_user['s_r'].sum()/source_grade_3_user.shape[0],4)
        source_run_7date_comrate_grade_3 = round(source_grade_3_user['common'].sum()/source_grade_3_user.shape[0],4)
        source_run_7date_rubrate_grade_3 = round(source_grade_3_user['rubbish'].sum()/source_grade_3_user.shape[0],4)
    for i in range(1,4):        
        exec("source_info_2_grade_%s_user['source_run_7date_bigrate'] = [source_run_7date_bigrate_grade_%s]"%(i,i))
        exec("source_info_2_grade_%s_user['source_run_7date_midrate'] = [source_run_7date_midrate_grade_%s]"%(i,i))
        exec("source_info_2_grade_%s_user['source_run_7date_minrate'] = [source_run_7date_minrate_grade_%s]"%(i,i))
        exec("source_info_2_grade_%s_user['source_run_7date_comrate'] = [source_run_7date_comrate_grade_%s]"%(i,i))
        exec("source_info_2_grade_%s_user['source_run_7date_rubrate'] = [source_run_7date_rubrate_grade_%s]"%(i,i))
    
    
    source_info_2_user = pd.concat([source_info_2_grade_1_user,source_info_2_grade_2_user,source_info_2_grade_3_user]).reset_index(drop=True)        
    
    
    
    # 计划成本、ROI维度
    source_info_2_grade_1 = pd.DataFrame(columns=['source_cost','standard_run_date','game_platform','source_run_date_cost','source_run_date_roi_1',
                                                      'source_run_date_roi_2','source_run_date_roi_3','source_run_date_roi_4','source_run_date_roi_5',
                                                      'source_run_date_roi_6','source_run_date_roi_7','source_run_date_predict_roi_30'])
    source_info_2_grade_2 = pd.DataFrame(columns=['source_cost','standard_run_date','game_platform','source_run_date_cost','source_run_date_roi_1',
                                                      'source_run_date_roi_2','source_run_date_roi_3','source_run_date_roi_4','source_run_date_roi_5',
                                                      'source_run_date_roi_6','source_run_date_roi_7','source_run_date_predict_roi_30'])
    source_info_2_grade_3 = pd.DataFrame(columns=['source_cost','standard_run_date','game_platform','source_run_date_cost','source_run_date_roi_1',
                                                      'source_run_date_roi_2','source_run_date_roi_3','source_run_date_roi_4','source_run_date_roi_5',
                                                      'source_run_date_roi_6','source_run_date_roi_7','source_run_date_predict_roi_30'])
    
    
    for i in range(1,8):
        standard_info_duplicates = standard_info_ios[standard_info_ios['role_data_type'] == i].drop_duplicates(['user_id','cp_server_no','cp_role_id'])
        # standard_info_duplicates = standard_info_duplicates[standard_info_duplicates['source_amount']!=0]
        
        source_grade_1 = standard_info_duplicates[standard_info_duplicates['source_amount']>=10000]
        source_grade_2 = standard_info_duplicates[(standard_info_duplicates['source_amount']<10000)&(standard_info_duplicates['source_amount']>=5000)]
        source_grade_3 = standard_info_duplicates[(standard_info_duplicates['source_amount']<5000)&(standard_info_duplicates['source_amount']>=500)]
        
        
        def dell(df):
            groups = df.groupby(['channel_id','source_id'])
            temp = groups['pay_sum'].agg([('pay_sum_nd','sum')]).reset_index()
            df = pd.merge(df,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')

            temp = groups['role_created_30_pay_sum'].agg([('source_30_pay_sum','sum')]).reset_index()
            df = pd.merge(df,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')

            temp = groups['user_id'].agg([('user_num','count')]).reset_index()
            df = pd.merge(df,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
            df = df[['channel_id','source_id','create_role_date','source_amount','game_platform',
                                                         'pay_sum_nd','source_30_pay_sum','user_num']].drop_duplicates()
            groups = df.groupby(['channel_id','source_id'])
            temp = groups['source_amount'].agg([('source_amount_sum','sum')]).reset_index()
            df = pd.merge(df,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
            df = df[['channel_id','source_id','source_amount_sum','game_platform',
                                                          'pay_sum_nd','source_30_pay_sum','user_num']].drop_duplicates()
            return df
        source_grade_1 = dell(source_grade_1)
        source_grade_2 = dell(source_grade_2)
        source_grade_3 = dell(source_grade_3)
                
        # grade_1
        if len(source_grade_1)!=0:             
            source_run_date_cost_grade_1 = round(source_grade_1[source_grade_1['user_num']!=0]['source_amount_sum'].sum()/source_grade_1[source_grade_1['user_num']!=0]['user_num'].sum(),2)            
            source_run_date_predict_roi_30_grade_1 = round(source_grade_1['source_30_pay_sum'].sum()/source_grade_1['source_amount_sum'].sum(),6)
            exec("source_run_date_roi_%s_grade_1 = round(source_grade_1['pay_sum_nd'].sum()/source_grade_1['source_amount_sum'].sum(),6)"%(i))
                      
        # grade_2
        if len(source_grade_2)!=0:
            source_run_date_cost_grade_2 = round(source_grade_2[source_grade_2['user_num']!=0]['source_amount_sum'].sum()/source_grade_2[source_grade_2['user_num']!=0]['user_num'].sum(),2)  
            source_run_date_predict_roi_30_grade_2 = round(source_grade_2['source_30_pay_sum'].sum()/source_grade_2['source_amount_sum'].sum(),6)
            exec("source_run_date_roi_%s_grade_2 = round(source_grade_2['pay_sum_nd'].sum()/source_grade_2['source_amount_sum'].sum(),6)"%(i))
         
        # grade_3
        if len(source_grade_3)!=0:
            source_run_date_cost_grade_3 = round(source_grade_3[source_grade_3['user_num']!=0]['source_amount_sum'].sum()/source_grade_3[source_grade_3['user_num']!=0]['user_num'].sum(),2)
            source_run_date_predict_roi_30_grade_3 = round(source_grade_3['source_30_pay_sum'].sum()/source_grade_3['source_amount_sum'].sum(),6)
            exec("source_run_date_roi_%s_grade_3 = round(source_grade_3['pay_sum_nd'].sum()/source_grade_3['source_amount_sum'].sum(),6)"%(i))
                                             
        source_info_2_grade_1['game_platform'] = [2]
        source_info_2_grade_1['source_run_date_cost'] = [source_run_date_cost_grade_1]
        exec("source_info_2_grade_1['source_run_date_roi_%s'] = [source_run_date_roi_%s_grade_1]"%(i,i))
        source_info_2_grade_1['source_run_date_predict_roi_30'] = [source_run_date_predict_roi_30_grade_1]
        source_info_2_grade_1['source_cost'] = ['>=10000']
        source_info_2_grade_1['standard_run_date'] = [datetime.date.today()]
   
        source_info_2_grade_2['game_platform'] = [2]
        source_info_2_grade_2['source_run_date_cost'] = [source_run_date_cost_grade_2]
        exec("source_info_2_grade_2['source_run_date_roi_%s'] = [source_run_date_roi_%s_grade_2]"%(i,i))
        source_info_2_grade_2['source_run_date_predict_roi_30'] = [source_run_date_predict_roi_30_grade_2]
        source_info_2_grade_2['source_cost'] = ['<10000 and >=5000']
        source_info_2_grade_2['standard_run_date'] = [datetime.date.today()]

        source_info_2_grade_3['game_platform'] = [2]
        source_info_2_grade_3['source_run_date_cost'] = [source_run_date_cost_grade_3]
        exec("source_info_2_grade_3['source_run_date_roi_%s'] = [source_run_date_roi_%s_grade_3]"%(i,i))
        source_info_2_grade_3['source_run_date_predict_roi_30'] = [source_run_date_predict_roi_30_grade_3]
        source_info_2_grade_3['source_cost'] = ['<5000 and >=500']
        source_info_2_grade_3['standard_run_date'] = [datetime.date.today()]
    
    source_info_2 = pd.concat([source_info_2_grade_1,source_info_2_grade_2,source_info_2_grade_3]).reset_index(drop=True)
    df = pd.concat([source_info_2,source_info_2_user], axis=1)
    return df

In [38]:
%%time
df_an = standard_compute_an(standard_info_an)

Wall time: 12.3 s


In [39]:
df_an

,source_cost,standard_run_date,game_platform,source_run_date_cost,source_run_date_roi_1,source_run_date_roi_2,source_run_date_roi_3,source_run_date_roi_4,source_run_date_roi_5,source_run_date_roi_6,source_run_date_roi_7,source_run_date_predict_roi_30,source_run_7date_bigrate,source_run_7date_midrate,source_run_7date_minrate,source_run_7date_comrate,source_run_7date_rubrate
0,>=5000,2020-08-29,1,19.75,0.066037,0.109233,0.143968,0.165771,0.185659,0.206392,0.222593,0.560729,0.0008,0.0050,0.0063,0.1853,0.8026
1,<5000 and >=3000,2020-08-29,1,17.70,0.047349,0.070782,0.093704,0.113242,0.127075,0.137495,0.149180,0.341269,0.0006,0.0037,0.0056,0.1847,0.8054
2,<3000 and >=500,2020-08-29,1,24.64,0.031648,0.053375,0.070582,0.083132,0.093480,0.105228,0.115073,0.283008,0.0005,0.0044,0.0062,0.2240,0.7649


In [40]:
%%time
df_ios = standard_compute_ios(standard_info_ios)

Wall time: 3.62 s


In [41]:
df_ios

,source_cost,standard_run_date,game_platform,source_run_date_cost,source_run_date_roi_1,source_run_date_roi_2,source_run_date_roi_3,source_run_date_roi_4,source_run_date_roi_5,source_run_date_roi_6,source_run_date_roi_7,source_run_date_predict_roi_30,source_run_7date_bigrate,source_run_7date_midrate,source_run_7date_minrate,source_run_7date_comrate,source_run_7date_rubrate
0,>=10000,2020-08-29,2,97.53,0.036685,0.067050,0.090637,0.108206,0.121699,0.140072,0.147302,0.421784,0.0030,0.0118,0.0121,0.5986,0.3744
1,<10000 and >=5000,2020-08-29,2,61.67,0.043787,0.079392,0.098066,0.119829,0.136564,0.156581,0.172368,0.465996,0.0022,0.0094,0.0089,0.5287,0.4508
2,<5000 and >=500,2020-08-29,2,55.11,0.039272,0.063950,0.080071,0.096123,0.108685,0.120426,0.128262,0.325475,0.0014,0.0065,0.0067,0.5154,0.4700


In [42]:
df = pd.concat([df_an,df_ios])

In [43]:
df

,source_cost,standard_run_date,game_platform,source_run_date_cost,source_run_date_roi_1,source_run_date_roi_2,source_run_date_roi_3,source_run_date_roi_4,source_run_date_roi_5,source_run_date_roi_6,source_run_date_roi_7,source_run_date_predict_roi_30,source_run_7date_bigrate,source_run_7date_midrate,source_run_7date_minrate,source_run_7date_comrate,source_run_7date_rubrate
0,>=5000,2020-08-29,1,19.75,0.066037,0.109233,0.143968,0.165771,0.185659,0.206392,0.222593,0.560729,0.0008,0.0050,0.0063,0.1853,0.8026
1,<5000 and >=3000,2020-08-29,1,17.70,0.047349,0.070782,0.093704,0.113242,0.127075,0.137495,0.149180,0.341269,0.0006,0.0037,0.0056,0.1847,0.8054
2,<3000 and >=500,2020-08-29,1,24.64,0.031648,0.053375,0.070582,0.083132,0.093480,0.105228,0.115073,0.283008,0.0005,0.0044,0.0062,0.2240,0.7649
0,>=10000,2020-08-29,2,97.53,0.036685,0.067050,0.090637,0.108206,0.121699,0.140072,0.147302,0.421784,0.0030,0.0118,0.0121,0.5986,0.3744
1,<10000 and >=5000,2020-08-29,2,61.67,0.043787,0.079392,0.098066,0.119829,0.136564,0.156581,0.172368,0.465996,0.0022,0.0094,0.0089,0.5287,0.4508
2,<5000 and >=500,2020-08-29,2,55.11,0.039272,0.063950,0.080071,0.096123,0.108685,0.120426,0.128262,0.325475,0.0014,0.0065,0.0067,0.5154,0.4700


In [45]:
df.to_csv('.standard_ios_an.csv')